# This is the notebook you can use to get the csv file containing all service requests between 2017 and 2019, along with their ElapsedTime and ElapsedDays.

In [2]:
import pandas as pd
import numpy as np 
from datetime import datetime
from datetime import date

In [9]:
#Adds ElaspedTime and ElapsedDays column. 


def convertTime(string):
    """Converts a string to a datetime object.
    """
    time = datetime.strptime(string,'%m/%d/%Y %I:%M:%S %p') 
    return time

def convertDays(string):
    """
    Converts string to date object. 
    """
    time = datetime.strptime(string[:10],'%m/%d/%Y') #Accepts time string, converts it to datetime object.
    return time

def convertFromSeconds(s): # total seconds
    """ convertFromSeconds(s): Converts an integer # of seconds into a list of [days, hours, minutes, seconds]
        input s: an int
    """
    s = s*60
    days = s // (24*60*60)  # total days
    s = s % (24*60*60) # remainder s
    hours = s // (60*60) # total hours
    s = s % (60*60) # remainder s
    minutes = s // 60 # total minutes
    s = s % 60 # remainder s
    statement = (days, ' days') + (hours, ' hrs') +(minutes, ' mins') + (s, 'sec')
    return statement
    
def elapsedTime(csv2017, csv2018, csv2019):
    """
    Accepts CSV files containing 2017-2019 service requests and creates new dataframe of all timestamps for both service request submission and fulfillment times.
    Input CSV file in the format: r'filepath'
    Data source: https://data.lacity.org/A-Well-Run-City/MyLA311-Service-Request-Data-2019/pvft-t768
    """
    df = pd.concat(map(pd.read_csv, [csv2017, csv2018, csv2019]), ignore_index = True)
    
    hdf = df.dropna(axis=0, subset=['CreatedDate', 'ClosedDate'])

    #ElapsedTime 
    df1 = hdf['ClosedDate'].apply(convertTime, 0)
    df2 = hdf['CreatedDate'].apply(convertTime, 0)
    
    hdf['ElapsedTime'] = df1 - df2  
    hdf['ElapsedTime'] = hdf['ElapsedTime']/np.timedelta64(1,'m') 
    hdf['ElapsedTime'] = hdf['ElapsedTime'].apply(convertFromSeconds, 0)   
    
    #ElapsedDays
    df3 = hdf['CreatedDate'].apply(convertDays, 0)
    df4 = hdf['ClosedDate'].apply(convertDays, 0) 
    hdf['ElapsedDays'] = (df4 - df3).dt.days
    
    #Column for Closed Dates
    hdf['Just Date'] = df3
    
    return hdf.reset_index(drop = True)


def elapsedTime2020(df):
    """
    Creates elapsedTime, elapsedDays, and date columns. 
    """
    #df = pd.read_csv(csv)
    
    hdf = df.dropna(axis=0, subset=['CreatedDate', 'ClosedDate'])

    #ElapsedTime 
    df1 = hdf['ClosedDate'].apply(convertTime, 0)   
    df2 = hdf['CreatedDate'].apply(convertTime, 0)
 
    hdf['ElapsedTime'] = df1 - df2
    hdf['ElapsedTime'] = hdf['ElapsedTime']/np.timedelta64(1,'m') 
    hdf['ElapsedTime'] = hdf['ElapsedTime'].apply(convertFromSeconds, 0)   
    
    #ElapsedDays
    df3 = hdf['CreatedDate'].apply(convertDays, 0)
    df4 = hdf['ClosedDate'].apply(convertDays, 0) 
    hdf['ElapsedDays'] = (df4 - df3).dt.days
    
    #Column for Closed Dates
    hdf['Just Date'] = df3
    
    return hdf.reset_index(drop = True).sort_index(axis=1)

def cleaned_df(csv): 
    """
    Take in 2020 dataset and return dataset with ElapsedTime, ElapsedDays, and date columns
    Data: https://data.lacity.org/A-Well-Run-City/MyLA311-Service-Request-Data-2020/rq3b-xjk8
    """
    df = pd.read_csv(csv) 
    df = df.drop(143589,axis=0).reset_index(drop = True)  #This row was mislabled (the year inputted was 3020, not 2020)
    edf = elapsedTime2020(df)
    edf = edf[edf.ElapsedDays >= 0]
    
    return edf


In [19]:
edf = elapsedTime('service2017.csv', 'service2018.csv', 'service2019.csv')
edf2 = cleaned_df('service2020.csv')

C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ngeta\AppData\Local\Continuum\a

In [20]:
edf = edf[edf['ElapsedDays'] < 1100]   #Some of the service requests had dates which were mislabelled, resulting in impossible ElapsedDay values. This command gets rid of those requests. 

In [17]:
d = pd.read_csv('service2020.csv')

C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
df_final = pd.concat([edf,edf2])

C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [26]:
edf

,APC,ActionTaken,Address,AddressVerified,Anonymous,ApproximateAddress,AssignTo,CD,CDMember,ClosedDate,...,StreetName,Suffix,TBMColumn,TBMPage,TBMRow,UpdatedDate,ZipCode,ElapsedTime,ElapsedDays,Just Date
0,West Los Angeles APC,SR Created,"222 N ASHDALE PL, 90049",Y,Y,N,NaN,5.0,Paul Koretz,01/01/2017 09:26:00 AM,...,ASHDALE,PL,H,631.0,1.0,01/01/2017 09:26:00 AM,90049,"(0.0, days, 9.0, hrs, 25.0, mins, 0.0, sec)",0,2017-01-01
1,East Los Angeles APC,SR Created,"1624 N ALLESANDRO ST, 90026",Y,Y,N,CCAC,13.0,Mitch O'Farrell,01/12/2017 05:10:00 PM,...,ALLESANDRO,ST,E,594.0,6.0,01/12/2017 05:10:00 PM,90026,"(11.0, days, 16.0, hrs, 25.0, mins, 0.0, sec)",11,2017-01-01
2,West Los Angeles APC,SR Created,"250 N ASHDALE AVE, 90049",Y,Y,N,NaN,5.0,Paul Koretz,01/01/2017 09:27:00 AM,...,ASHDALE,AVE,H,631.0,1.0,01/01/2017 09:27:00 AM,90049,"(0.0, days, 8.0, hrs, 41.0, mins, 0.0, sec)",0,2017-01-01
3,North Valley APC,SR Created,"21824 W DEVONSHIRE ST, 91311",Y,N,N,WVA,12.0,Mitchell Englander,01/04/2017 05:33:00 PM,...,DEVONSHIRE,ST,A,500.0,4.0,01/04/2017 05:33:00 PM,91311,"(3.0, days, 16.0, hrs, 32.0, mins, 0.0, sec)",3,2017-01-01
4,North Valley APC,SR Created,"21230 W DEVONSHIRE ST, 91311",Y,N,N,WVA,12.0,Mitchell Englander,01/04/2017 05:34:00 PM,...,DEVONSHIRE,ST,B,500.0,4.0,01/04/2017 05:34:00 PM,91311,"(3.0, days, 16.0, hrs, 25.0, mins, 0.0, sec)",3,2017-01-01
5,East Los Angeles APC,SR Created,"2630 E CARLETON AVE, 90065",Y,Y,N,NEGB_ELA,1.0,Gilbert Cedillo,01/03/2017 06:55:00 PM,...,CARLETON,AVE,H,594.0,5.0,01/03/2017 06:55:00 PM,90065,"(2.0, days, 17.0, hrs, 39.0, mins, 0.0, sec)",2,2017-01-01
6,Central APC,SR Created,"747 N SEWARD ST, 90038",Y,N,NaN,NC,4.0,David Ryu,01/07/2017 09:46:00 AM,...,SEWARD,ST,E,593.0,6.0,01/07/2017 09:46:00 AM,90038,"(6.0, days, 8.0, hrs, 25.0, mins, 0.0, sec)",6,2017-01-01
7,East Los Angeles APC,SR Created,"4850 N ELLENWOOD DR, 90041",Y,N,NaN,NC,14.0,Jose Huizar,01/04/2017 09:31:00 AM,...,ELLENWOOD,DR,J,564.0,6.0,01/04/2017 09:31:00 AM,90041,"(3.0, days, 8.0, hrs, 5.0, mins, 0.0, sec)",3,2017-01-01
8,North Valley APC,SR Created,"10236 N TOPANGA CANYON BLVD, 91311",Y,N,NaN,WV,12.0,Mitchell Englander,01/04/2017 09:51:00 AM,...,TOPANGA CANYON,BLVD,A,500.0,4.0,01/04/2017 09:51:00 AM,91311,"(3.0, days, 8.0, hrs, 20.0, mins, 0.0, sec)",3,2017-01-01
9,Central APC,SR Created,"3814 W COUNTRY CLUB DR, 90019",Y,N,NaN,NC,10.0,Herb J. Wesson Jr.,01/07/2017 08:53:00 AM,...,COUNTRY CLUB,DR,G,633.0,4.0,01/07/2017 08:53:00 AM,90019,"(6.0, days, 7.0, hrs, 6.0, mins, 0.0, sec)",6,2017-01-01


In [27]:
df_final.tail()

,APC,ActionTaken,Address,AddressVerified,Anonymous,ApproximateAddress,AssignTo,CD,CDMember,ClosedDate,...,SRNumber,ServiceDate,Status,StreetName,Suffix,TBMColumn,TBMPage,TBMRow,UpdatedDate,ZipCode
1061511,South Valley APC,SR Created,"5755 N WILKINSON AVE, 91607",Y,N,N,EV,2.0,Paul Krekorian,09/29/2020 07:05:15 AM,...,1-1749237311,09/29/2020 12:00:00 AM,Closed,WILKINSON,AVE,F,562.0,1.0,09/29/2020 07:08:20 AM,91607
1061512,South Los Angeles APC,SR Created,"1062 W 56TH ST, 90037",Y,N,N,SLA,9.0,Curren D. Price Jr.,09/23/2020 03:57:34 PM,...,1-1749238151,09/23/2020 12:00:00 AM,Closed,56TH,ST,A,674.0,5.0,09/23/2020 03:58:49 PM,90037
1061513,North Valley APC,SR Created,"10435 N WHEATLAND AVE, 91040",Y,N,NaN,EV,7.0,Monica Rodriguez,09/28/2020 05:28:33 PM,...,1-1749237341,09/28/2020 12:00:00 AM,Closed,WHEATLAND,AVE,B,503.0,3.0,09/28/2020 05:30:18 PM,91040
1061514,Central APC,SR Created,"3945 W 8TH ST, 90005",Y,N,N,NC,4.0,David Ryu,09/25/2020 07:11:02 AM,...,1-1749236441,09/25/2020 12:00:00 AM,Closed,8TH,ST,H,633.0,3.0,09/25/2020 07:17:30 AM,90005
1061515,NaN,SR Created,"8801 N SNOWDEN AVE, 91331",Y,N,N,EV,NaN,NaN,10/01/2020 07:52:14 AM,...,1-1749236501,10/01/2020 12:00:00 AM,Closed,SNOWDEN,AVE,NaN,NaN,NaN,10/02/2020 05:27:13 AM,91331


In [28]:
df_final.to_csv('2020service.csv',index=False)

In [5]:
edf.to_csv('fservice.csv', index = False)

In [6]:
df = pd.read_csv('fservice.csv')

C:\Users\ngeta\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.head()

,APC,ActionTaken,Address,AddressVerified,Anonymous,ApproximateAddress,AssignTo,CD,CDMember,ClosedDate,...,StreetName,Suffix,TBMColumn,TBMPage,TBMRow,UpdatedDate,ZipCode,ElapsedTime,ElapsedDays,Just Date
0,West Los Angeles APC,SR Created,"222 N ASHDALE PL, 90049",Y,Y,N,NaN,5.0,Paul Koretz,01/01/2017 09:26:00 AM,...,ASHDALE,PL,H,631.0,1.0,01/01/2017 09:26:00 AM,90049,"(0.0, ' days', 9.0, ' hrs', 25.0, ' mins', 0.0...",0,2017-01-01
1,East Los Angeles APC,SR Created,"1624 N ALLESANDRO ST, 90026",Y,Y,N,CCAC,13.0,Mitch O'Farrell,01/12/2017 05:10:00 PM,...,ALLESANDRO,ST,E,594.0,6.0,01/12/2017 05:10:00 PM,90026,"(11.0, ' days', 16.0, ' hrs', 25.0, ' mins', 0...",11,2017-01-01
2,West Los Angeles APC,SR Created,"250 N ASHDALE AVE, 90049",Y,Y,N,NaN,5.0,Paul Koretz,01/01/2017 09:27:00 AM,...,ASHDALE,AVE,H,631.0,1.0,01/01/2017 09:27:00 AM,90049,"(0.0, ' days', 8.0, ' hrs', 41.0, ' mins', 0.0...",0,2017-01-01
3,North Valley APC,SR Created,"21824 W DEVONSHIRE ST, 91311",Y,N,N,WVA,12.0,Mitchell Englander,01/04/2017 05:33:00 PM,...,DEVONSHIRE,ST,A,500.0,4.0,01/04/2017 05:33:00 PM,91311,"(3.0, ' days', 16.0, ' hrs', 32.0, ' mins', 0....",3,2017-01-01
4,North Valley APC,SR Created,"21230 W DEVONSHIRE ST, 91311",Y,N,N,WVA,12.0,Mitchell Englander,01/04/2017 05:34:00 PM,...,DEVONSHIRE,ST,B,500.0,4.0,01/04/2017 05:34:00 PM,91311,"(3.0, ' days', 16.0, ' hrs', 25.0, ' mins', 0....",3,2017-01-01
